# Question 04
Find the passengers who have been on more than N flights together within the range (from,to).

## Assumptions
1. "more than 3 flights together" means 4, 5, ... which is bigger than 3. 
2. Data is clearned and not errorneous
3. Timezone consideration is not required

## Approaches

1. SQL self join that matches with non-self passengerId that have the same flightId.
2. Matrix M * M<sup>T</sup>
Self product matrix Matrix M * M<sup>T</sup> has diagonal represents the number of flghts of respective passenger, and right top part (row/passenger 1, column/passenger2) represents how many flights passenger2 shares with passenger 1.

### TODO
Implement a matrix way. 

## Output

To avoid having duplicates, passegner 1 ID < passenger ID 2. For instance there will be no (0, 1, 14) and (1, 0, 14) togeher.

# Setup

In [1]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

### Spark parition control based on core availability

In [2]:
val NUM_CORES = 4
val NUM_PARTITIONS = 3

lazy val spark: SparkSession = SparkSession.builder()
    .master("local")
    .appName("flight")
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)

import spark.implicits._

NUM_CORES = 4
NUM_PARTITIONS = 3
spark = <lazy>


<lazy>

## Constants

In [3]:
val CSV_DELIMITER = ","
val FLIGHTDATA_CSV_PATH = "../resources/flightData.csv"
val PASSENGER_CSV_PATH = "../resources/passengers.csv"

val DATE_FORMAT = "yyyy-MM-dd"
val FLIGHT_DATE_FROM = "2017-01-01"
val FLIGHT_DATE_TO   = "2017-12-31"
val NUM_FLIGHT_TOGETHER = 3

val RESULT_DIR = "results/flightsTogether"

CSV_DELIMITER = ,
FLIGHTDATA_CSV_PATH = ../resources/flightData.csv
PASSENGER_CSV_PATH = ../resources/passengers.csv
DATE_FORMAT = yyyy-MM-dd
FLIGHT_DATE_FROM = 2017-01-01
FLIGHT_DATE_TO = 2017-12-31
NUM_FLIGHT_TOGETHER = 3
RESULT_DIR = results/flightsTogether


results/flightsTogether

# Tools

### Elapsed time profiler

In [4]:
import scala.collection.mutable.ListBuffer

val timing = new StringBuffer
val times = new ListBuffer[Long]()

def clear(): Unit = {
    timing.setLength(0)
    times.clear
}
def average(): Long = {
    times.reduce(_+_) / times.length
}

/**
@param label Description about the run
@code code to execute
@return execution
*/
def timed[T](label: String, code: => T): T = {
    val start = System.currentTimeMillis()
    val result = code
    val stop = System.currentTimeMillis()
    timing.append(s"Processing $label took ${stop - start} ms.\n")
    times.append(stop - start)
    result
}

timing = 
times = ListBuffer()


clear: ()Unit
average: ()Long
timed: [T](label: String, code: => T)T


ListBuffer()

In [5]:
// To flush out error: missing argument list for method timed
println("")

<console>:45: error: missing argument list for method timed
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `timed _` or `timed(_,_)` instead of `timed`.
       timed
       ^
lastException: Throwable = null


### Save to file

In [6]:
def save(df: DataFrame) = {
    df.coalesce(1)
    .write
    .format("csv")
    .mode(SaveMode.Overwrite)
    .option("header", "true")
    .save(RESULT_DIR)
}

save: (df: org.apache.spark.sql.DataFrame)Unit


### Spark SQL runner

# Main

## Base DataFrame

In [7]:
// Transformations, no action yet
val flightData = spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("dateFormat", "yyyy-MM-dd")
    .option("inferSchema", "true")
    .load("../resources/flightData.csv")
    .select(
        "passengerId",
        "flightId",
        "date"
    )

flightData = [passengerId: int, flightId: int ... 1 more field]


[passengerId: int, flightId: int ... 1 more field]

## SQL

In [8]:
val queryFlightsTogether = s"""
WITH 
    --------------------------------------------------------------------------------
    -- Passengers flew more than NUM_FLIGHT_TOGETHER times.
    --------------------------------------------------------------------------------
    more_than_n_flights AS (
        SELECT passengerId
        FROM flightData
        GROUP BY passengerId
        --------------------------------------------------------------------------------
        -- "More than 3" means 4, 5, 6, ... not including 3.
        --------------------------------------------------------------------------------
        HAVING count(flightId) > $NUM_FLIGHT_TOGETHER
        ORDER BY passengerId
    )

SELECT 
    f.passengerId AS `Passenger 1 ID`, 
    s.passengerId AS `Passenger 2 ID`, 
    count(s.flightId) AS `Number of flights together`,
    '$FLIGHT_DATE_FROM' as From,
    '$FLIGHT_DATE_TO' as To
FROM
    flightData f 
    --------------------------------------------------------------------------------
    -- Passengers more than NUM_FLIGHT_TOGETHER flights
    --------------------------------------------------------------------------------
    INNER JOIN more_than_n_flights m 
        ON f.passengerId == m.passengerId
    --------------------------------------------------------------------------------
    -- Passengers who shared same flights
    --------------------------------------------------------------------------------
    INNER JOIN flightData s 
        ON f.flightId == s.flightId
WHERE
    --------------------------------------------------------------------------------
    -- Only have (passenger 1 > passenger 2) to avoid having both (p1, p2) and (p2, p1) 
    --------------------------------------------------------------------------------
    f.passengerId < s.passengerId AND  
    f.date >= to_timestamp('$FLIGHT_DATE_FROM', '$DATE_FORMAT') AND
    f.date <= to_timestamp('$FLIGHT_DATE_TO',   '$DATE_FORMAT') AND
    s.date >= to_timestamp('$FLIGHT_DATE_FROM', '$DATE_FORMAT') AND
    s.date <= to_timestamp('$FLIGHT_DATE_TO',   '$DATE_FORMAT')
GROUP BY 
    f.passengerId, s.passengerId
HAVING 
    count(s.flightId) > $NUM_FLIGHT_TOGETHER
--ORDER BY 
--    `Number of flights together`, 
--    f.passengerId, 
--    s.passengerId
"""

queryFlightsTogether = 


"
WITH
    --------------------------------------------------------------------------------
    -- Passengers flew more than NUM_FLIGHT_TOGETHER times.
    --------------------------------------------------------------------------------
    more_than_n_flights AS (
        SELECT passengerId
        FROM flightData
        GROUP BY passengerId
        --------------------------------------------------------------------------------
        -- "More than 3" means 4, 5, 6, ... not including 3.
        --------------------------------------------------------------------------------
        HAVING count(flightId) > 3
        ORDER BY passengerId
    )
SELECT
    f.passengerId AS `Passenger 1 ID`,
    s.passengerId AS `Passenger 2 ID`,
    count(s.flightId) AS ...


In [9]:
/**
Run the SparkSQL 
@param label Lable to describe this run
@param query SQL 
@param repeats Number of run
@return Average execution time in msec
*/
def run(label: String, repeats: Int, toSave: Boolean = false): Long = {
    val result = spark
        .sql(queryFlightsTogether)
        .sort(
//            desc("Number of flights together"), 
            asc("Passenger 1 ID"),
            asc("Passenger 2 ID")
        )

    clear()
    for (i <- (0 until repeats)){
        timed(
            label,
            result.show(3)
        )
        println(timing)
        println(s"Average time $average ms")
    }
    println(result.rdd.toDebugString)    

    if(toSave) save(result)
    
    average
}

run: (label: String, repeats: Int, toSave: Boolean)Long


## Order by (passengerId)

In [10]:
var df = flightData
    .orderBy("passengerId")
    .persist
    .createOrReplaceTempView("flightData")

val timeOrderPassenger = run(
    "Order by (passengerId)",
    3,
    true
)
spark.catalog.dropTempView("flightData")

flightData.unpersist

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|             1|            37|                         4|2017-01-01|2017-12-31|
|             1|            38|                         4|2017-01-01|2017-12-31|
|             1|            76|                         4|2017-01-01|2017-12-31|
+--------------+--------------+--------------------------+----------+----------+
only showing top 3 rows

Processing Order by (passengerId) took 4898 ms.

Average time 4898 ms
+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|             1|            37|                         4|2017-01-01|2017-12-31|
|             

timeOrderPassenger = 3033


df: Unit = ()


[passengerId: int, flightId: int ... 1 more field]

## Order by (flightId)

In [11]:
var df = flightData
    .orderBy("flightId")
    .persist
    .createOrReplaceTempView("flightData")

val timeOrderFlight = run(
    "Order by (flightId)",
    3
)

spark.catalog.dropTempView("flightData")
flightData.unpersist

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|          2717|          2759|                        14|2017-01-01|2017-12-31|
|          3503|          3590|                        14|2017-01-01|2017-12-31|
+--------------+--------------+--------------------------+----------+----------+
only showing top 3 rows

Processing Order by (flightId) took 5463 ms.

Average time 5463 ms
+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|          2717| 

timeOrderFlight = 4114


df: Unit = ()


[passengerId: int, flightId: int ... 1 more field]

## Order by ("passengerId", "flightId")

In [12]:
var df = flightData
    .orderBy("passengerId", "flightId")
    .persist
    .createOrReplaceTempView("flightData")

val timeOrderPassengerFlight= run(
    "Order by (passengerId, flightId)",
    3
)

spark.catalog.dropTempView("flightData")
flightData.unpersist

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|          2717|          2759|                        14|2017-01-01|2017-12-31|
|          3503|          3590|                        14|2017-01-01|2017-12-31|
+--------------+--------------+--------------------------+----------+----------+
only showing top 3 rows

Processing Order by (passengerId, flightId) took 4354 ms.

Average time 4354 ms
+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|   

timeOrderPassengerFlight = 3467


df: Unit = ()


[passengerId: int, flightId: int ... 1 more field]

## Order by ("passengerId", "date")¶

In [13]:
var df = flightData
    .orderBy("passengerId", "date")
    .persist
    .createOrReplaceTempView("flightData")

val timeOrderPassengerDate = run(
    "Order by (passengerId, date)",
    3
)

spark.catalog.dropTempView("flightData")
flightData.unpersist

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|          2717|          2759|                        14|2017-01-01|2017-12-31|
|          3503|          3590|                        14|2017-01-01|2017-12-31|
+--------------+--------------+--------------------------+----------+----------+
only showing top 3 rows

Processing Order by (passengerId, date) took 4251 ms.

Average time 4251 ms
+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|       

timeOrderPassengerDate = 3442


df: Unit = ()


[passengerId: int, flightId: int ... 1 more field]

## Order by none

In [14]:
flightData
    .persist
    .createOrReplaceTempView("flightData")

val timeOrderNone = run(
    "Order by none",
    3
)

spark.catalog.dropTempView("flightData")
flightData.unpersist

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|          2717|          2759|                        14|2017-01-01|2017-12-31|
|          3503|          3590|                        14|2017-01-01|2017-12-31|
+--------------+--------------+--------------------------+----------+----------+
only showing top 3 rows

Processing Order by none took 5774 ms.

Average time 5774 ms
+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|          2717|       

timeOrderNone = 3982


[passengerId: int, flightId: int ... 1 more field]

## Elepased Time

In [15]:
val report = s"""
Order by:
Non                     is $timeOrderNone ms
(passengerId)           is $timeOrderPassenger ms
(flightId)              is $timeOrderFlight ms
(passengerId, date)     is $timeOrderPassengerDate ms
(passengerId, flightId) is $timeOrderPassengerFlight ms
"""
println(report)


Order by:
Non                     is 3982 ms
(passengerId)           is 3052 ms
(flightId)              is 4114 ms
(passengerId, date)     is 3442 ms
(passengerId, flightId) is 3467 ms



report = 


"
Order by:
Non                     is 3982 ms
(passengerId)           is 3052 ms
(flightId)              is 4114 ms
(passengerId, date)     is 3442 ms
(passengerId, flightId) is 3467 ms
"


# Validations

In [16]:
flightData.createOrReplaceTempView("flightData")

In [17]:
var query = 
s"""
SELECT passengerId, count(flightId) as count
FROM flightData
WHERE passengerId in (1, 37, 38)
GROUP BY passengerId
HAVING count(flightId) > $NUM_FLIGHT_TOGETHER
ORDER BY passengerId
"""
spark.sql(query)
.show()

query = 
s"""
SELECT passengerId, flightId
FROM flightData
WHERE passengerId in (1, 37, 38)
ORDER BY passengerId
"""
spark.sql(query)
.show()

+-----------+-----+
|passengerId|count|
+-----------+-----+
|          1|    5|
|         37|    4|
|         38|    4|
+-----------+-----+

+-----------+--------+
|passengerId|flightId|
+-----------+--------+
|          1|       0|
|          1|     972|
|          1|     901|
|          1|     993|
|          1|     940|
|         37|     972|
|         37|       0|
|         37|     940|
|         37|     901|
|         38|       0|
|         38|     901|
|         38|     940|
|         38|     972|
+-----------+--------+



query = 
query = 


"
SELECT passengerId, flightId
FROM flightData
WHERE passengerId in (1, 37, 38)
ORDER BY passengerId
"
SELECT passengerId, flightId
FROM flightData
WHERE passengerId in (1, 37, 38)
ORDER BY passengerId


### Random samples

In [18]:
val seed = 42
val withReplacement = false
val fraction = 0.01
spark.sql(queryFlightsTogether)
    .sample(withReplacement, fraction, seed)
.show()

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|             1|            37|                         4|2017-01-01|2017-12-31|
|           618|           648|                         5|2017-01-01|2017-12-31|
|           704|           749|                         5|2017-01-01|2017-12-31|
|           712|          1043|                         4|2017-01-01|2017-12-31|
|           366|           759|                         6|2017-01-01|2017-12-31|
|          1878|          1891|                         5|2017-01-01|2017-12-31|
|          2165|          2181|                         4|2017-01-01|2017-12-31|
|          3532|          3552|                         4|2017-01-01|2017-12-31|
|           246|          3195|                         4|2017-01-01|2017-12-31|
|          2510|          31

seed = 42
withReplacement = false
fraction = 0.01


0.01

In [19]:
"""
comm -12 \
<(cat ../../../main/resources/flightData.csv | awk '{FS=","} /^1381,/{print $2}' | sort ) \
<(cat ../../../main/resources/flightData.csv | awk '{FS=","} /^58,/{print $2}' | sort)

131
189
217
247
272
283
331

comm -12 \
<(cat ../../../main/resources/flightData.csv | awk '{FS=","} /^58,/{print $2}' | sort ) \
<(cat ../../../main/resources/flightData.csv | awk '{FS=","} /^2942,/{print $2}' | sort)

131
189
217
247
$ comm -12 \
> <(cat ../../../main/resources/flightData.csv | awk '{FS=","} /^701,/{print $2}' | sort ) \
> <(cat ../../../main/resources/flightData.csv | awk '{FS=","} /^760,/{print $2}' | sort)
13
18
333
34
361
374
391
404
432
45
58
7
77
91
96
"""

val filter01 = col("Passenger 2 ID") === 2942
val filter02 = col("Passenger 2 ID") === 1381
spark.sql(queryFlightsTogether)
    .where(col("Passenger 1 ID") === 58)
    .where(filter01.or(filter02))
    .show()

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|            58|          1381|                         7|2017-01-01|2017-12-31|
|            58|          2942|                         4|2017-01-01|2017-12-31|
+--------------+--------------+--------------------------+----------+----------+



filter01 = (Passenger 2 ID = 2942)
filter02 = (Passenger 2 ID = 1381)


<console>:44: warning: a pure expression does nothing in statement position; you may be omitting necessary parentheses
"""
^


(Passenger 2 ID = 1381)

In [20]:
flightData.select(
    countDistinct("passengerId"),
    min("passengerId"),
    max("passengerId")
).show

flightData.select(
    countDistinct("flightId"),
    min("flightId"),
    max("flightId")
).show

val queryPassengersPerFlight = """
SELECT flightId, count(passengerId) as count
FROM flightData
GROUP BY flightId
ORDER BY count DESC
"""
spark.sql(queryPassengersPerFlight)
    .show()

val totalPassengers = """
SELECT count(passengerId) as total_passengers
FROM flightData
"""
spark.sql(totalPassengers)
    .show()

+---------------------------+----------------+----------------+
|count(DISTINCT passengerId)|min(passengerId)|max(passengerId)|
+---------------------------+----------------+----------------+
|                      15500|               1|           15500|
+---------------------------+----------------+----------------+

+------------------------+-------------+-------------+
|count(DISTINCT flightId)|min(flightId)|max(flightId)|
+------------------------+-------------+-------------+
|                    1000|            0|          999|
+------------------------+-------------+-------------+

+--------+-----+
|flightId|count|
+--------+-----+
|      14|  100|
|      18|  100|
|      25|  100|
|      38|  100|
|      46|  100|
|      50|  100|
|      73|  100|
|      97|  100|
|     161|  100|
|     172|  100|
|     186|  100|
|     225|  100|
|     232|  100|
|     233|  100|
|     248|  100|
|     254|  100|
|     257|  100|
|     263|  100|
|     280|  100|
|     282|  100|
+--------+--

queryPassengersPerFlight = 
totalPassengers = 


"
SELECT flightId, count(passengerId) as count
FROM flightData
GROUP BY flightId
ORDER BY count DESC
"
"
SELECT count(passengerId) as total_passengers
FROM flightData
"


In [21]:
spark.catalog.dropTempView("flightData")

true